# Модели стохастических объектов (методы анализа данных) 
## Практическая работа №1 (Генерация выборки)
### КИ18-16 Прекель В.А.

#### Используется библиотека FsHttp для http-запросов и JsonProvider для доступа к json-данным в статически-типизированном стиле

In [1]:
#r "nuget: FSharp.Data, 3.3.3"
#r "nuget: SchlenkR.FsHttp, 5.0.0"

open System
open System.IO
open FSharp.Data
open FsHttp
open FsHttp.DslCE
open System.Text.Json

#### JsonProvider выводит типы из вручную скачанного фрагмента

In [1]:
type json = JsonProvider<"sample.json">

#### Количество комментариев под 5 постом в фрагменте:

In [1]:
json.GetSample().Response.Items.[4].Likes.Count

290

#### ServiceToken для доступа к Vk Api

In [1]:
let accessToken = "1bb9ca221bb9ca221bb9ca22ad1bdfa76e11bb91bb9ca22441bbfc7d2cfe35c00c4a071"

#### Функции асинхронного получения и парсинга `count` постов со смещениев в `offset` постов

In [1]:
let getAsync (count: int) (offset: int) =
    httpAsync {
        GET $"https://api.vk.com/method/wall.get?v=5.78&owner_id=-120075923&access_token={accessToken}&count={count}&offset={offset}"
    }
let getItemsAsync count offset =
    async {
        let! q = getAsync count offset

        return json.Load(q.content.ReadAsStream()).Response.Items
    }


#### Сколько лайков под последними 4 постами:

In [1]:
let y = getItemsAsync 5 0 |> Async.RunSynchronously |> Seq.map (fun i -> i.Likes.Count)
y



index,value
0,299
1,531
2,267
3,407
4,326


#### Структура нужных данных и функция преобразования в нужные данные

In [1]:
type Post =
    { OwnerId: int
      Id: int
      Date: DateTimeOffset
      AttachmentTypes: string list
      Comments: int
      Likes: int
      Reposts: int
      Views: Nullable<int>
      FromId: int
      PostSource: string
      PostType: string
      SignerId: Nullable<int>
      MarkedAsAds: int }

let responseToPost (r: json.Item) =
      { OwnerId = r.OwnerId
        Id = r.Id
        Date = DateTimeOffset.FromUnixTimeSeconds(int64 r.Date)
        AttachmentTypes =
            r.Attachments
            |> Array.map (fun t -> t.Type)
            |> List.ofArray
        Comments = r.Comments.Count
        Likes = r.Likes.Count
        Reposts = r.Reposts.Count
        Views = Option.toNullable (r.Views |> Option.map (fun v -> v.Count))
        FromId = r.FromId
        PostSource = r.PostSource.Type
        PostType = r.PostType 
        SignerId = Option.toNullable r.SignerId
        MarkedAsAds = r.MarkedAsAds }


#### Кол-во постов

In [1]:
let count =
    json
        .Load((getAsync 0 0 |> Async.RunSynchronously)
            .content.ReadAsStream())
        .Response.Count
count

38890

#### Сколько требуется сделать запросов и с какими смещениями, еспри принимать по 100 постов:

In [1]:
let queries = (float <| count) / 100. |> ceil |> int

let offsets = [ 0 .. queries ] |> List.map ((*) 100)
let offsets_fst = [ 0 .. queries ] |> List.map ((*) 100) |> List.head |> List.singleton
(queries, offsets)

Item1,Item2
389,"[ 0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900 ... (370 more) ]"


#### Получение всех постов и преобразование в нужную структуру паралелльно со степенью параллелизма 4

In [1]:
let items =
    offsets
    |> List.map (getItemsAsync 100) // Для всех смещений получается 100 постов
    // |> (fun v -> Async.Parallel(v, 4)) // 4 одновременно. Чтобы максимально параллельно закомментировать эту строку и раскомментировать следующую
    |> Async.Parallel 
    |> Async.RunSynchronously // Запустить это синхронно
    |> Array.concat // Склеить результат
    |> Array.map responseToPost // Преобразовать в нужную структуру
    |> Array.sortBy (fun p -> p.Date) // Отсортировтаь по дате
    |> Array.distinctBy (fun p -> p.Id) // Посты должны быть с уникальмими id, поэтому исключить дублирующиеся если они как-то появились 


In [1]:
items

index,OwnerId,Id,Date,AttachmentTypes,Comments,Likes,Reposts,Views,FromId,PostSource,PostType,SignerId,MarkedAsAds
0,-120075923,1,2016-04-23 18:49:13Z,[ photo ],28,61,1,<null>,-120075923,api,post,<null>,0
1,-120075923,2,2016-04-23 19:18:26Z,[ photo ],1,122,2,<null>,-120075923,api,post,<null>,0
2,-120075923,3,2016-04-23 19:35:32Z,[ photo ],0,225,9,<null>,-120075923,api,post,<null>,0
3,-120075923,4,2016-04-23 19:35:39Z,[ photo ],9,38,0,<null>,-120075923,api,post,<null>,0
4,-120075923,7,2016-04-23 20:24:46Z,[ ],9,45,0,<null>,-120075923,vk,post,<null>,0
5,-120075923,23,2016-04-24 11:57:00Z,"[ photo, photo, photo, photo, photo, photo, photo, photo, photo ]",6,57,0,<null>,-120075923,api,post,<null>,0
6,-120075923,37,2016-04-24 13:42:00Z,"[ photo, photo, photo, photo, photo, photo, photo, photo, photo, photo ]",2,395,22,<null>,-120075923,vk,post,<null>,0
7,-120075923,40,2016-04-24 15:22:23Z,"[ photo, photo ]",7,23,0,<null>,-120075923,api,post,<null>,0
8,-120075923,52,2016-04-24 17:32:04Z,[ photo ],4,41,2,<null>,-120075923,vk,post,<null>,0
9,-120075923,61,2016-04-25 05:30:37Z,[ photo ],3,35,0,<null>,-120075923,api,post,<null>,0


#### Сохранение в файл

In [1]:
let out = new StreamWriter "memeblog.json"
fprintfn out "%s" (JsonSerializer.Serialize <| items)
out.Close()
